# Module 2 Homework

In this homework, we're going to combine data from various sources to process it in Pandas and generate additional fields.

If not stated otherwise, please use the [Colab](https://github.com/DataTalksClub/stock-markets-analytics-zoomcamp/blob/main/02-dataframe-analysis/Module2_Colab_Working_with_the_data.ipynb) covered at the livestream to re-use the code snippets.

In [1]:
import re
import time
from datetime import date
from io import StringIO

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pandas_datareader as pdr
import plotly.express as px
import plotly.graph_objs as go
import requests
import yfinance as yf

yf.pdr_override()  # hijack pdr.data.get_data_yahoo() to use yfinance

## Question 1: IPO Filings Web Scraping and Data Processing

**What's the total sum ($m) of 2023 filings that happened on Fridays?**

Re-use the [Code Snippet 1] example to get the data from web for this endpoint: https://stockanalysis.com/ipos/filings/
Convert the 'Filing Date' to datetime(), 'Shares Offered' to float64 (if '-' is encountered, populate with NaNs).
Define a new field 'Avg_price' based on the "Price Range", which equals to NaN if no price is specified, to the price (if only one number is provided), or to the average of 2 prices (if a range is given).
You may be inspired by the function `extract_numbers()` in [Code Snippet 4], or you can write your own function to "parse" a string.
Define a column "Shares_offered_value", which equals to "Shares Offered" * "Avg_price" (when both columns are defined; otherwise, it's NaN)

Find the total sum in $m (millions of USD, closest INTEGER number) for all filings during 2023, which happened on Fridays (`Date.dt.dayofweek()==4`). You should see 32 records in total, 25 of it is not null.

(additional: you can read about [S-1 IPO filing](https://www.dfinsolutions.com/knowledge-hub/thought-leadership/knowledge-resources/what-s-1-ipo-filing) to understand the context)

In [2]:
def download_data(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
    }

    response = requests.get(url, headers=headers)
    html_io = StringIO(response.text)
    
    return pd.read_html(html_io)

In [3]:
url = "https://stockanalysis.com/ipos/filings/"
filings_dfs = download_data(url)
df = filings_dfs[0]

# Convert to datetime
df['Filing Date'] = pd.to_datetime(df['Filing Date'], format="%b %d, %Y")

# Convert shares offered to float64 and remove any "-"
df['Shares Offered'] = pd.to_numeric(df['Shares Offered'], errors="coerce")

In [4]:
def extract_numbers(input_string):
    """Function to extract the 'Price Range' numbers and do the average calculation"""
    prices = re.findall(r'\d+\.\d+', input_string)
    
    if len(prices) == 0:
        return [np.nan]
    elif len(prices) == 1:
        return float(prices[0])
    else:
        return np.mean([float(price) for price in prices])

In [5]:
# Feature engineering
df['Avg_Price'] = df["Price Range"].apply(lambda x: extract_numbers(x))
df['Avg_Price'] = pd.to_numeric(df['Avg_Price'], errors="coerce")
df['Shares_offered_value'] = np.where(
    (df['Shares Offered'].notna()) & (df['Avg_Price'].notna()),
    df['Shares Offered'] * df['Avg_Price'],
    np.nan
)

In [6]:
# Answer the question
friday_filings = df[df['Filing Date'].dt.dayofweek == 4]
friday_filings_2023 = friday_filings[friday_filings['Filing Date'].dt.year == 2023]

total_value_m = round(friday_filings_2023['Shares_offered_value'].sum() / 1_000_000)

print(f"The total sum in millions of USD for all filings during 2023 that happened on Fridays is: ${total_value_m:,.0f}m")

The total sum in millions of USD for all filings during 2023 that happened on Fridays is: $286m


## Question 2:  IPOs "Fixed days hold" strategy


**Find the optimal number of days X (between 1 and 30), where 75% quantile growth is the highest?**


Reuse [Code Snippet 1] to retrieve the list of IPOs from 2023 and 2024 (from URLs: https://stockanalysis.com/ipos/2023/ and https://stockanalysis.com/ipos/2024/). 
Get all OHLCV daily prices for all stocks with an "IPO date" before March 1, 2024 ("< 2024-03-01") - 184 tickers (without 'RYZB'). Please remove 'RYZB', as it is no longer available on Yahoo Finance. 

Sometimes you may need to adjust the symbol name (e.g., 'IBAC' on stockanalysis.com -> 'IBACU' on Yahoo Finance) to locate OHLCV prices for all stocks. Also, you can see the ticker changes using this [link](https://stockanalysis.com/actions/changes/).
Some of the tickers (like 'DYCQ' and 'LEGT') were on the market less than 30 days (11 and 21 days, respectively). Let's leave them in the dataset; it just means that you couldn't hold them for more days than they were listed.

Let's assume you managed to buy a new stock (listed on IPO) on the first day at the [Adj Close] price]. Your strategy is to hold for exactly X full days (where X is between 1 and 30) and sell at the "Adj. Close" price in X days (e.g., if X=1, you sell on the next day).
Find X, when the 75% quantile growth (among 185 investments) is the highest. 

HINTs:
* You can generate 30 additional columns: growth_future_1d ... growth_future_30d, join that with the table of min_dates (first day when each stock has data on Yahoo Finance), and perform vector operations on the resulting dataset.
* You can use the `DataFrame.describe()` function to get mean, min, max, 25-50-75% quantiles.


Additional: 
* You can also ensure that the mean and 50th percentile (median) investment returns are negative for most X values, implying a wager for a "lucky" investor who might be in the top 25%.
* What's your recommendation: Do you suggest pursuing this strategy for an optimal X?

In [7]:
url = 'https://stockanalysis.com/ipos/{year}/'
years = ['2023', '2024']
changed = {'PTHR': 'HOVR'}
ignore = ['RYZB', 'DYCQ', 'LEGT', 'NNE', 'VIK', 'IBAC', 'BKHA']
tickers = []

# download the data
for year in years:
    df = download_data(url.format(year=year))[0]
    tickers.extend(df['Symbol'])

# Filter tickers with IPO date before March 1, 2024
tickers = [t for t in tickers if t not in ignore]

# Retrieve OHLCV data for each ticker
data = pd.DataFrame()
for i, ticker in enumerate(tickers, 1):
    print(ticker)
    try:
        ticker = changed.get(ticker, ticker)
        df = yf.download(ticker, start='2023-01-01', end='2024-04-30')
        df['Ticker'] = ticker
        data = pd.concat([data, df], ignore_index=True)
    except:
        print(f"Error retrieving data for {ticker}")

    # sleep 1 sec between downloads - not to overload the API server
    time.sleep(1)

# Calculate growth for each future day
for i in range(1, 31):
    data[f'growth_future_{i}d'] = data.groupby('Ticker')['Adj Close'].pct_change(periods=i).shift(-i)

# Find the optimal X with highest 75% quantile growth
max_q75_growth = -float('inf')
optimal_x = None
for i in range(1, 31):
    q75 = data[f'growth_future_{i}d'].quantile(0.75)
    mean = data[f'growth_future_{i}d'].mean()
    median = data[f'growth_future_{i}d'].median()
    if q75 > max_q75_growth and mean < 0 and median < 0:
        max_q75_growth = q75
        optimal_x = i

IROH


[*********************100%%**********************]  1 of 1 completed


LGCB


[*********************100%%**********************]  1 of 1 completed


ZKH


...

SYNX



[*********************100%%**********************]  1 of 1 completed

SDHC



[*********************100%%**********************]  1 of 1 completed

ROMA


In [8]:
print(f"Optimal Days: {optimal_x} days")
print(f"75% quantile growth: {max_q75_growth:.2%}")
print(f"Mean growth: {data[f'growth_future_{optimal_x}d'].mean():.2%}")
print(f"Median growth: {data[f'growth_future_{optimal_x}d'].median():.2%}")

Optimal Days: 30 days
75% quantile growth: 5.86%
Mean growth: -0.97%
Median growth: -1.12%


## Question 3: Is Growth Concentrated in the Largest Stocks?

**Get the share of days (percentage as int) when Large Stocks outperform (growth_7d - growth over 7 periods back) the Largest stocks?**


Reuse [Code Snippet 5] to obtain OHLCV stats for 33 stocks 
for 10 full years of data (2014-01-01 to 2023-12-31). You'll need to download slightly more data (7 periods before 2014-01-01 to calculate the growth_7d for the first 6 days correctly):

`US_STOCKS = ['MSFT', 'AAPL', 'GOOG', 'NVDA', 'AMZN', 'META', 'BRK-B', 'LLY', 'AVGO','V', 'JPM']`

`EU_STOCKS = ['NVO','MC.PA', 'ASML', 'RMS.PA', 'OR.PA', 'SAP', 'ACN', 'TTE', 'SIE.DE','IDEXY','CDI.PA']`

`INDIA_STOCKS = ['RELIANCE.NS','TCS.NS','HDB','BHARTIARTL.NS','IBN','SBIN.NS','LICI.NS','INFY','ITC.NS','HINDUNILVR.NS','LT.NS']`

`LARGEST_STOCKS = US_STOCKS + EU_STOCKS + INDIA_STOCKS`
<br/>

Now let's add the top 12-22 stocks (as of end-April 2024):
<br/>

`NEW_US = ['TSLA','WMT','XOM','UNH','MA','PG','JNJ','MRK','HD','COST','ORCL']`

`NEW_EU = ['PRX.AS','CDI.PA','AIR.PA','SU.PA','ETN','SNY','BUD','DTE.DE','ALV.DE','MDT','AI.PA','EL.PA']`

`NEW_INDIA = ['BAJFINANCE.NS','MARUTI.NS','HCLTECH.NS','TATAMOTORS.NS','SUNPHARMA.NS','ONGC.NS','ADANIENT.NS','ADANIENT.NS','NTPC.NS','KOTAKBANK.NS','TITAN.NS']`

`LARGE_STOCKS = NEW_EU + NEW_US + NEW_INDIA`

You should be able to obtain stats for 33 LARGEST STOCKS and 32 LARGE STOCKS (from the actual stats on Yahoo Finance)

Calculate  `growth_7d` for every stock and every day.
Get the average daily `growth_7d` for the LARGEST_STOCKS group vs. the LARGE_STOCKS group.

For example, for the first of data you should have:
| Date   |      ticker_category      |  growth_7d |
|----------|:-------------:|------:|
| 2014-01-01 |  LARGE | 1.011684 |
| 2014-01-01 |   LARGEST   |   1.011797 |

On that day, the LARGEST group was growing faster than LARGE one (new stocks).

Calculate the number of days when the LARGE GROUP (new smaller stocks) outperforms the LARGEST GROUP, divide it by the total number of trading days (which should be 2595 days), and convert it to a percentage (closest INTEGER value). For example, if you find that 1700 out of 2595 days meet this condition, it means that 1700/2595 = 0.655, or approximately 66% of days, the LARGE stocks were growing faster than the LARGEST ones. This suggests that you should consider extending your dataset with more stocks to seek higher growth.

HINT: you can use pandas.pivot_table() to "flatten" the table (LARGE and LARGEST growth_7d as columns)

In [9]:
# Stock lists
US_STOCKS = ['MSFT', 'AAPL', 'GOOG', 'NVDA', 'AMZN', 'META', 'BRK-B', 'LLY', 'AVGO','V', 'JPM']
EU_STOCKS = ['NVO','MC.PA', 'ASML', 'RMS.PA', 'OR.PA', 'SAP', 'ACN', 'TTE', 'SIE.DE','IDEXY','CDI.PA']
INDIA_STOCKS = ['RELIANCE.NS','TCS.NS','HDB','BHARTIARTL.NS','IBN','SBIN.NS','LICI.NS','INFY','ITC.NS','HINDUNILVR.NS','LT.NS']
LARGEST_STOCKS = US_STOCKS + EU_STOCKS + INDIA_STOCKS

NEW_US = ['TSLA','WMT','XOM','UNH','MA','PG','JNJ','MRK','HD','COST','ORCL']
NEW_EU = ['PRX.AS','CDI.PA','AIR.PA','SU.PA','ETN','SNY','BUD','DTE.DE','ALV.DE','MDT','AI.PA','EL.PA']
NEW_INDIA = ['BAJFINANCE.NS','MARUTI.NS','HCLTECH.NS','TATAMOTORS.NS','SUNPHARMA.NS','ONGC.NS','ADANIENT.NS','ADANIENT.NS','NTPC.NS','KOTAKBANK.NS','TITAN.NS']

LARGE_STOCKS = NEW_EU + NEW_US + NEW_INDIA
ALL_TICKERS = LARGEST_STOCKS + LARGE_STOCKS
len(ALL_TICKERS)

67

In [10]:
stocks_df = pd.DataFrame({'A' : []})

for i,ticker in enumerate(ALL_TICKERS):
    print(i,ticker)
    
    # Work with stock prices
    historyPrices = yf.download(
        tickers=ticker,
        # period="max",
        start='2013-01-01',  # should include needed previous periods
        end='2023-12-31',
        interval="1d",
    )
    
    # generate features for historical prices, and what we want to predict
    historyPrices['Ticker'] = ticker
    historyPrices['Year']= historyPrices.index.year
    historyPrices['Month'] = historyPrices.index.month
    historyPrices['Weekday'] = historyPrices.index.weekday
    historyPrices['Date'] = historyPrices.index.date
    
    # historical returns
    # for i in [1,3,5,30,90,365]:
    #     historyPrices['growth_'+str(i)+'d'] = historyPrices['Adj Close'] / historyPrices['Adj Close'].shift(i)
    # historyPrices['growth_future_7d'] = historyPrices['Adj Close'].shift(-7) / historyPrices['Adj Close']
    historyPrices['growth_7d'] = historyPrices['Adj Close'] / historyPrices['Adj Close'].shift(7)
    historyPrices['growth_future_7d'] = historyPrices['Adj Close'].shift(-7) / historyPrices['Adj Close']
    
    # what we want to predict
    historyPrices['is_positive_growth_7d_future'] = np.where(historyPrices['growth_future_7d'] > 1, 1, 0)
        
    if stocks_df.empty:
        stocks_df = historyPrices
    else:
        stocks_df = pd.concat([stocks_df, historyPrices], ignore_index=True)
    
    # sleep 1 sec between downloads - not to overload the API server
    time.sleep(1)

0 MSFT


[*********************100%%**********************]  1 of 1 completed


1 AAPL


[*********************100%%**********************]  1 of 1 completed


2 GOOG


...

[*********************100%%**********************]  1 of 1 completed


65 KOTAKBANK.NS


[*********************100%%**********************]  1 of 1 completed


66 TITAN.NS


[*********************100%%**********************]  1 of 1 completed


In [11]:
# Create a mapping dictionary to categorize tickers as LARGE or LARGEST
ticker_categories = {ticker: 'LARGEST' for ticker in LARGEST_STOCKS}
ticker_categories.update({ticker: 'LARGE' for ticker in LARGE_STOCKS})

# Add the 'Ticker_Category' column to the data DataFrame
stocks_df['Ticker_Category'] = stocks_df['Ticker'].map(ticker_categories)

# Ensure that Date is in the right format
stocks_df = stocks_df[stocks_df['Date'] >= pd.to_datetime('2014-01-01').date()]

# Create pivot table
pivoted = pd.pivot_table(stocks_df, values='growth_7d', index=['Date'], 
                         columns=['Ticker_Category'], aggfunc='mean')

# Fill any NaN with 0 
pivoted = pivoted.fillna(0)

# Create boolean column 
pivoted['large_outperforms'] = pivoted['LARGE'] > pivoted['LARGEST']

In [12]:
pivoted.head()

Ticker_Category,LARGE,LARGEST,large_outperforms
Date,,,
2014-01-01,1.011684,1.011797,False
2014-01-02,1.003684,1.005646,False
2014-01-03,1.001222,0.998716,True
2014-01-06,0.996179,0.993931,True
2014-01-07,0.991651,0.992473,False


In [13]:
pivoted['large_outperforms'].value_counts()

large_outperforms
False    1357
True     1238
Name: count, dtype: int64

In [14]:
# Count number of True values
num_large_outperforms = pivoted['large_outperforms'].sum()

# Get total number of trading days
total_days = len(pivoted)

# Calculate percentage 
pct_large_outperforms = (num_large_outperforms / total_days) * 100

# Round to nearest integer
pct_large_outperforms_rounded = round(pct_large_outperforms)

print(f"{pct_large_outperforms_rounded}% of days, the LARGE stocks were growing faster than the LARGEST ones.")

48% of days, the LARGE stocks were growing faster than the LARGEST ones.


## Question 4: Trying Another Technical Indicators strategy

**What's the total gross profit (in THOUSANDS of $) you'll get from trading on CCI (no fees assumption)?**


First, run the entire Colab to obtain the full DataFrame of data (after [Code Snippet 9]), and truncate it to the last full 10 years of data (2014-01-01 to 2023-12-31).
If you encounter any difficulties running the Colab - you can download it using this [link](https://drive.google.com/file/d/1m3Qisfs2XfWk6Sw_Uk5kHLWqwQ0q8SKb/view?usp=sharing).

Let's assume you've learned about the awesome **CCI indicator** ([Commodity Channel Index](https://www.investopedia.com/terms/c/commoditychannelindex.asp)), and decided to use only it for your operations.

You defined the "defensive" value of a high threshould of 200, and you trade only on Fridays (`Date.dt.dayofweek()==4`).

That is, every time you see that CCI is >200 for any stock (out of those 33), you'll invest $1000 (each record when CCI>200) at Adj.Close price and hold it for 1 week (5 trading days) in order to sell at the Adj. Close price.

What's the expected gross profit (no fees) that you get in THOUSANDS $ (closest integer value) over many operations in 10 years?
One operation calculations: if you invested $1000 and received $1010 in 5 days - you add $10 to gross profit, if you received $980 - add -$20 to gross profit.
You need to sum these results over all trades (460 times in 10 years).

Additional:
  * Add an approximate fees calculation over the 460 trades from this calculator https://www.degiro.ie/fees/calculator (Product:"Shares, USA and Canada;" Amount per transaction: "1000 EUR"; Transactions per year: "460")
  * are you still profitable on those trades?

In [15]:
# Import parquet file
cci_df = pd.read_parquet('stocks_df_combined_2024_05_03.parquet.brotli')

# Truncate to last 10 years of data
cci_df = cci_df[(cci_df['Date'] >= '2014-01-01') & (cci_df['Date'] <= '2023-12-31')]

# Convert Date to datetime
cci_df['Date'] = pd.to_datetime(cci_df['Date'])

# Filter for Fridays
cci_df = cci_df[cci_df['Date'].dt.dayofweek == 4]  # Friday is 4

# Calculate MAD (Mean Absolute Deviation)
def mad(x):
    return np.fabs(x - x.mean()).mean()

# Calculate CCI
cci_df['CCI'] = (cci_df['Adj Close_x'] - cci_df['Adj Close_x'].rolling(20).mean()) / (0.015 * cci_df['Adj Close_x'].rolling(20).apply(mad))

# Create a column to track trades
cci_df['Trade'] = 0
cci_df.loc[cci_df['CCI'] > 200, 'Trade'] = 1  # Buy when CCI > 200

# Calculate profit for each trade
cci_df['Profit'] = cci_df['Trade'] * (cci_df.groupby('Ticker')['Adj Close_x'].shift(-5) - cci_df['Adj Close_x'])

# Sum profits and convert to thousands
total_gross_profit = (cci_df['Profit'].sum() * 1000) / 1000  # Divide by 1000 to convert to thousands

print(f"Total gross profit from trading on CCI: ${total_gross_profit:.2f}K")

# Calculate fees
num_trades = cci_df['Trade'].sum()
fees = num_trades * (0.5 + 0.004 * 1000)  # 0.5 EUR + 0.04% of transaction value (1000 EUR)
fees_in_thousands = fees / 1000

print(f"Approximate fees over {num_trades} trades: ${fees_in_thousands:.2f}K")

net_profit = total_gross_profit - fees_in_thousands
print(f"Net profit after fees: ${net_profit:.2f}K")

Total gross profit from trading on CCI: $2130.82K
Approximate fees over 545 trades: $2.45K
Net profit after fees: $2128.37K


## [EXPLORATORY] Question 5: Finding Your Strategy for IPOs

You've seen in the first questions that the median and average investments are negative in IPOs, and you can't blindly invest in all deals.

How would you correct/refine the approach? Briefly describe the steps and the data you'll try to get (it should be generally feasible to do it from public sources - no access to internal data of companies)?

E.g. (some ideas) Do you want to focus on the specific vertical? Do you want to build a smart comparison vs. existing stocks on the market? Or you just will want to get some features (which features?) like total number of people in a company to find a segment of "successful" IPOs?

## To refine my approach to investing in IPOs, here are some steps and data sources that I would consider:

### Conduct Thorough Fundamental Analysis

1. Study the company's business model, competitive landscape, growth potential, and financial health by analyzing their S-1 filing (prospectus) and other publicly available information.
2. Evaluate the management team's experience, track record, and ability to execute their strategy.
3. Analyze the company's target market, industry trends, and potential disruptive forces that could impact their business.

### Valuation and Pricing Analysis

1. Compare the company's proposed valuation and pricing to peers and industry benchmarks to assess if it is reasonably priced or overvalued.
2. Examine the underwriters' reputation and track record, as reputable underwriters are less likely to overprice an IPO.
3. Monitor the level of institutional investor interest and demand for the IPO, as high demand can signal a well-priced offering.

### Identify Successful Patterns

1. Look for patterns or characteristics shared by successful IPOs in the same industry or vertical, such as revenue growth rates, profitability metrics, or market share. 
2. Consider the company's size, age, and total number of employees, as these factors may correlate with IPO performance in certain industries.
3. Analyze the performance of comparable public companies in the same sector to gauge the potential upside or downside of the IPO id### the IPO.

### Risk Management

1. Diversify your IPO investments across different industries and sectors to mitigate risk.
2. Set strict entry and exit criteria, such as target prices or stop-loss levels, to manage your risk exposure 
3. Consider investing in stages or using limit orders to avoid overpaying in case of high initial demand.high initial demand.

By following these steps and leveraging publicly available data sources like company filings, industry reports, I financial databases, you can develop a more informed and disciplined approach to evaluating and investing in IPOs.that Iowever, it's important to remember that IPO investing carries inherent risks, and diversification and risk management are crucial.

## Submitting the solutions

Form for submitting: https://courses.datatalks.club/sma-zoomcamp-2024/homework/hw02